<div style="border:solid Chocolate 2px; padding: 40px">

Гузель привет, меня зовут Станислав, и я буду проверять твой проект. Предлагаю общаться на «ты», но если это неудобно - дай знать, и мы перейдем на «вы».
Это учебный проект и здесь не стоит бояться делать ошибки, я хочу поделиться своим опытом и помочь тебе стать аналитиком данных.
Уже выполнена большая работа над проектом, давай сделаем его еще лучше. Ниже ты найдешь мои комментарии - пожалуйста, не перемещай, не изменяй и не удаляй их. Я буду использовать цветовую разметку:

<div class="alert alert-danger">
❌ <b>Комментарий с замечанием ревьюера:</b> Так выделены самые важные замечания. Как только ты их исправишь проект будет принят. </div>

<div class="alert alert-warning">
⚠️  <b>Комментарий с небольшим замечанием или советом ревьюера:</b> Так выделены небольшие замечания или советы. 
</div>

<div class="alert alert-success">
✔️ <b>Комментарий с комплиментом ревьюера:</b> Так я выделяю все остальные комментарии.</div>

Давай работать над проектом в диалоге: если ты что-то меняешь в проекте или отвечаешь на мои комментарии — пиши об этом.
Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>🎓 Комментарий студента:</b> Например, вот так.</div>
Всё это поможет выполнить повторную проверку твоего проекта оперативнее.
</div>


# Исследование лояльности клиентов телекоммуникационной компании

**Описание проекта**

Заказчиком исследования стала большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности (NPS) среди клиентов из России. Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

**Цель исследования** - проанализировать результаты опроса и построить дашборд, который представит информацию о текущем уровне NPS среди клиентов и покажет, как этот уровень меняется в зависимости от пользовательских признаков. 

**Описание данных:**

Данные взяты из SQLite. Путь к файлу - */datasets/telecomm_csi.db*

Переменные:

*Таблица user*:

- user_id - Идентификатор клиента, первичный ключ таблицы
- lt_day - Количество дней «жизни» клиента
- age - Возраст клиента в годах
- gender_segment - Пол клиента (1 – женщина, 0 – мужчина)
- os_name - Тип операционной системы
- cpe_type_name - Тип устройства
- location_id - Идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location
- age_gr_id - Идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment
- tr_gr_id - Идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице 
- lt_gr_id - Идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице lifetime_segment
- nps_score - Оценка клиента в NPS-опросе (от 1 до 10)

*Таблица location*:

- location_id - Идентификатор записи, первичный ключ
- country - Страна
- city - Город

*Таблица age_segment*:

- age_gr_id - Идентификатор сегмента, первичный ключ
- bucket_min - Минимальная граница сегмента
- bucket_max - Максимальная граница сегмента
- title - Название сегмента

*Таблица traffic_segment*:

- tr_gr_id - Идентификатор сегмента, первичный ключ
- bucket_min - Минимальная граница сегмента
- bucket_max - Максимальная граница сегмента
- title - Название сегмента

*Таблица lifetime_segment*:

- lt_gr_id - Идентификатор сегмента, первичный ключ
- bucket_min - Минимальная граница сегмента
- bucket_max - Максимальная граница сегмента
- title - Название сегмента

**План исследования:**

1. Подключение к базе
2. Выгрузка данных
3. Исследование распределения участников
4. Исследование лояльности пользователей
5. Составление портрета типичного сторонника компании
6. Создание дашборда в Tableau
7. Создание презентации
8. Общий вывод

## Подключение к базе

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## Выгрузка данных

Для создания дашборда выгрузим данные, собрав их из разных таблиц. Для определения лайфтайма клиента (времени жизни в использовании услуг) возьмем условие, что клиента считают новым, если количество дней «жизни» составляет не более 365 дней. А для составления категории по оценкам NPS используем следующее условие: 

- 9-10 баллов — «cторонники» 
- 7-8 баллов — «нейтралы» 
- 0-6 баллов — «критики» 

In [3]:
data = """
    SELECT u.user_id,
           u.lt_day,
           CASE
               WHEN u.lt_day <= 365 THEN 'новый'
               ELSE 'старый'
           END AS is_new,
           u.age,
           CASE
               WHEN u.gender_segment = 1 THEN 'женщина'
               WHEN u.gender_segment = 0 THEN 'мужчина'
               ELSE 'нет данных'
           END AS gender_segment,
           u.os_name,
           u.cpe_type_name,
           l.country,
           l.city,
           a.title AS age_segment,
           t.title AS traffic_segment,
           lt.title AS lifetime_segment,
           u.nps_score,
           CASE
               WHEN u.nps_score <= 6 THEN 'критики'
               WHEN u.nps_score <= 8 THEN 'нейтралы'
               ELSE 'cторонники'
           END AS nps_group
    FROM user AS u
    LEFT JOIN location AS l ON l.location_id = u.location_id
    LEFT JOIN age_segment AS a ON a.age_gr_id = u.age_gr_id
    LEFT JOIN traffic_segment AS t ON t.tr_gr_id = u.tr_gr_id
    LEFT JOIN lifetime_segment AS lt ON lt.lt_gr_id = u.lt_gr_id;
    """          

In [4]:
df = pd.read_sql(data, engine)
df

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старый,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,cторонники
1,A001WF,2344,старый,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,cторонники
2,A003Q7,467,старый,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,cторонники
3,A004TB,4190,старый,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,cторонники
4,A004XT,1163,старый,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,cторонники
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502488,ZZZKLD,1249,старый,54.0,женщина,ANDROID,SMARTPHONE,Россия,Москва,05 45-54,04 1-5,08 36+,5,критики
502489,ZZZLWY,129,новый,31.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,03 25-34,04 1-5,04 4-6,8,нейтралы
502490,ZZZQ5F,522,старый,36.0,мужчина,ANDROID,SMARTPHONE,Россия,Сургут,04 35-44,09 25-30,06 13-24,10,cторонники
502491,ZZZQ8E,2936,старый,37.0,женщина,ANDROID,SMARTPHONE,Россия,УланУдэ,04 35-44,17 65-70,08 36+,9,cторонники


In [5]:
df.describe()

,lt_day,age,nps_score
count,502493.000000,501939.000000,502493.000000
mean,1868.841439,39.621946,7.508562
std,1683.701762,11.188249,3.020378
min,-21.000000,10.000000,1.000000
25%,533.000000,31.000000,5.000000
50%,1239.000000,38.000000,9.000000
75%,3064.000000,47.000000,10.000000
max,9162.000000,89.000000,10.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [16]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

<div class="alert alert-success">
✔️ <b>Комментарий с комплиментом ревьюера:</b> 

Данные выгружены корректно, молодец. 
    
Можно вывести более полную статистику по датафрейму, применив describe() и info().

</div>


## Исследование распределения участников опроса

В опросе приняли участие клиенты от 10 до 89 лет. Большую часть составил сегмент от 35 до 44 лет (33,02%). Затем сегмент от 25 до 34 лет (30,81%) и от 45 до 54 лет (18,76%). Также в опросе поучаствовали в большинстве женщины (54,22%), чем мужчины (45,52%). Больше всего опрошенных оказалось в Москве (27,36%), затем в Санкт-Петербурге (12,25%) и Новосибирске (11%). В топ 10 городов по количеству опрошенных вошли также Екатеринбург, Казань, Нижний Новгород, Ростов-на-Дону, Самара, Омск, Челябинск.
Новых пользователей оказалось 86100 человек (17,13%), а старых гораздо больше - 416393 (82,87%).

## Исследование лояльности пользователей

Общий NPS пользователей оказался положительным 22%. Большая часть пользователей относятся к группе сторонников компании - 264081 пользователь (52,55%), чем к группе критиков - 154044 (30,66%). Нейтралы составили 16,79% - 84368 пользователей. Среди сторонников чаще встречаются клиенты от 35 до 44 лет (33%), а среди критиков клиенты от 25 до 34 лет (35,8%). Так как в опросе приняли участие в основном женщины, то они преобладают во всех категориях - сторонников (57%), критиков (52%) и нейтралов (51%). Чем больше время жизни клиента в компании, тем больше количество как сторонников сервиса, так и критиков. Критиков и сторонников больше при трафике 1-5. 

## Исследование сторонников компании

К сторонникам компании можно отнести пользователей, которые уже продолжительное время пользуются услугами сервиса (36+), им от 35 до 44 лет, чаще всего они потребляют трафик в ссегменте 1-5. 

## Создание дашборда в Tableau

На основании построенных графиков был создан единый дашборд

## Создание презентации

Также была создана презентация со всеми полученными выводами

Презентация находится по ссылке: https://public.tableau.com/shared/KHGSQDN33?:display_count=n&:origin=viz_share_link

## Общий вывод

Мы провели исследование лояльности клиентов телекоммуникационной компании

Целью исследования было - проанализировать результаты опроса и построить дашборд, который представит информацию о текущем уровне NPS среди клиентов и покажет, как этот уровень меняется в зависимости от пользовательских признаков.

Этапы исследования и выводы:

1. Подключение к базе

Мы подключились к базе данных в SQLite для того, чтобы выгрузить данные по пользователям и результатам опроса.

2. Выгрузка данных

Мы написали запрос на sql для того, чтобы выгрузить нужные нам данные, собрав их из разных таблиц. Для определения лайфтайма клиента (времени жизни в использовании услуг) возьмем условие, что клиента считают новым, если количество дней «жизни» составляет не более 365 дней. А для составления категории по оценкам NPS используем следующее условие:
- 9-10 баллов — «cторонники»
- 7-8 баллов — «нейтралы»
- 0-6 баллов — «критики»

3. Исследование распределения участников

В опросе приняли участие клиенты от 10 до 89 лет. Большую часть составил сегмент от 35 до 44 лет (33,02%). Затем сегмент от 25 до 34 лет (30,81%) и от 45 до 54 лет (18,76%). Также в опросе поучаствовали в большинстве женщины (54,22%), чем мужчины (45,52%). Больше всего опрошенных оказалось в Москве (27,36%), затем в Санкт-Петербурге (12,25%) и Новосибирске (11%). В топ 10 городов по количеству опрошенных вошли также Екатеринбург, Казань, Нижний Новгород, Ростов-на-Дону, Самара, Омск, Челябинск. Новых пользователей оказалось 86100 человек (17,13%), а старых гораздо больше - 416393 (82,87%).

4. Исследование лояльности пользователей

Общий NPS пользователей оказался положительным 22%. Большая часть пользователей относятся к группе сторонников компании - 264081 пользователь (52,55%), чем к группе критиков - 154044 (30,66%). Нейтралы составили 16,79% - 84368 пользователей. Среди сторонников чаще встречаются клиенты от 35 до 44 лет (33%), а среди критиков клиенты от 25 до 34 лет (35,8%). Так как в опросе приняли участие в основном женщины, то они преобладают во всех категориях - сторонников (57%), критиков (52%) и нейтралов (51%). Чем больше время жизни клиента в компании, тем больше количество как сторонников сервиса, так и критиков. Критиков и сторонников больше при трафике 1-5.

5. Составление портрета типичного сторонника компании

К сторонникам компании можно отнести пользователей, которые уже продолжительное время пользуются услугами сервиса (36+), им от 35 до 44 лет, чаще всего они потребляют трафик в сегменте 1-5.

6. Создание дашборда в Tableau
7. Создание презентации



<div class="alert alert-success">
✔️ <b>Комментарий с комплиментом ревьюера по Дашборду:</b>

    
Отличный анализ групп по пользователям, добавлены графики по основным категориям: полу, полу/возрасту, городам, выведено соотношение новых и старых пользователей, добавлены все необходимые фильтры.
    
Здорово, что сделан анализ по группам и дополнительно проанализированны сторонники.
    
Добавлен подробный анализ по нескольким разрезам.
</div>


<div class="alert alert-danger">
❌ <b>Комментарий с замечанием ревьюера по Дашборду:</b> 

Есть одно замечание, которое необходимо скорректировать:    


    
- добавь, пожалуйста, анализ уровня лояльности - NPS по городам, к примеру заказчику будет интересно узнать город с наименьшим или с наибольшим NPS, чтобы корректнее продумать стратегию
    

</div>



<div class="alert alert-info"> <b>🎓 Приятно познакомиться! Спасибо за комментарии) Добавила таблицу NPS в разных городах в дашборд и лист "Лояльность пользователей". Ссылка та же:
    
https://public.tableau.com/shared/2CK5CN4BD?:display_count=n&:origin=viz_share_link </div>

<div class="alert alert-success">
✔️ <b>Комментарий с комплиментом ревьюера по Дашборду в2:</b>

    
Отлично, спасибо за доработку дашборда!